In [1]:
import torch

In [2]:
print(torch.__version__)

1.3.0


### 基本数学运算

In [3]:
# 定义向量
vector = torch.tensor([1,2,3,4])
print('Vector:\t\t', vector)
print('Vector Shape:\t', vector.shape)

Vector:		 tensor([1, 2, 3, 4])
Vector Shape:	 torch.Size([4])


In [6]:
# 定义矩阵
matrix = torch.tensor([[1,2],[3,4]])
print('Matrix:\n', matrix)
print('Matrix Shape:\n', matrix.shape)

Matrix:
 tensor([[1, 2],
        [3, 4]])
Matrix Shape:
 torch.Size([2, 2])


In [7]:
# 定义张量
tensor = torch.tensor([ [[1,2],[3,4]],[[1,2],[3,4]] ])
print('Tensor:\n', tensor)
print('Tensor Shape:\n', tensor.shape)

Tensor:
 tensor([[[1, 2],
         [3, 4]],

        [[1, 2],
         [3, 4]]])
Tensor Shape:
 torch.Size([2, 2, 2])


### autograd

In [8]:
x = torch.ones(2,2, requires_grad = True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [9]:
y = x+2
z = y*y + 3
out = z.mean()
print(y)
print(z)
print(out)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[12., 12.],
        [12., 12.]], grad_fn=<AddBackward0>)
tensor(12., grad_fn=<MeanBackward0>)


In [10]:
out.backward()

In [11]:
print(x.grad)

tensor([[1.5000, 1.5000],
        [1.5000, 1.5000]])


In [13]:
with torch.no_grad():
    k = x + 1
    print(k)

tensor([[2., 2.],
        [2., 2.]])


### 简易神经网络

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
# 定义神经网络的结构 
class Net(nn.Module):
    def __init__(self): #定义神经网络结构，输入数据 1*32*32 （一个频道的图片）
        super(Net, self).__init__()
        # 第一层 卷积层 输入频道1，输出频道6，卷积3*3
        self.conv1 = nn.Conv2d(1,6,3)
        # 第二层 卷积层 输入频道6，输出频道16，卷积3*3
        self.conv2 = nn.Conv2d(6,16,3)
        # 第三层 全连接层 输入维度16*（32-2-2）*（32-2*2），输出维度512 
        self.fc1 = nn.Linear(16*28*28, 512)
        # 第四层 全连接层 输入维度512， 输出维度64
        self.fc2 = nn.Linear(512,64)
        # 第五层 全连接层 输入维度64， 输出维度2
        self.fc3 = nn.Linear(64,2)
    # 定义数据流向
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        
        x = x.view(-1, 16*28*28)
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.relu(x)       

        x = self.fc3(x)
        return x 

In [16]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=12544, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)


In [17]:
# 生成随机输入
input_data = torch.randn(1,1,32,32)
print(input_data)
print(input_data.size())

tensor([[[[ 0.0538, -1.1829,  0.6292,  ..., -0.2008,  2.0629,  0.0507],
          [-3.6702,  0.3457,  1.0183,  ..., -0.2030,  0.2603,  1.3901],
          [ 0.2676, -0.0787, -0.9515,  ..., -0.3650, -0.4259,  1.8939],
          ...,
          [-1.0673, -2.0431,  0.3687,  ..., -0.9156, -1.2444, -0.2700],
          [-0.4933, -0.5091, -2.0136,  ..., -2.5142,  0.0432,  0.8797],
          [ 0.9163,  0.7108, -1.0788,  ...,  0.1581,  0.3442,  0.5434]]]])
torch.Size([1, 1, 32, 32])


In [18]:
# 运行神经网络
out = net(input_data)
print(out)
print(out.size())

tensor([[-0.0050, -0.0123]], grad_fn=<AddmmBackward>)
torch.Size([1, 2])


In [19]:
# 随机生成Target
target = torch.randn(2)
target = target.view(1,-1)
print(target)

tensor([[-0.1658, -0.2020]])


In [20]:
# 定义损失函数
criterion = nn.L1Loss()
loss = criterion(out, target)
print(loss)

tensor(0.1753, grad_fn=<L1LossBackward>)


In [21]:
# 反向传递
# 清零所有梯度
net.zero_grad()
loss.backward() # 自动计算梯度，反向传递

In [23]:
# 更新权重， 定义更新规则 
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr= 0.01)
optimizer.step()

In [24]:
# 再次运行神经网络
out = net(input_data)
print(out)
print(out.size())

tensor([[-0.0572, -0.0621]], grad_fn=<AddmmBackward>)
torch.Size([1, 2])


In [25]:
loss = criterion(out, target) # 第二次的损失更小
print(loss) 

tensor(0.1243, grad_fn=<L1LossBackward>)
